In [4]:
'''
Need to determine how many animals have been tested for each gentoype.
Load the Summary1.xlsx where each sheet has the summary for each experiment.
Load the SummarizingResponses.xlsx that contains a list of each of each cross:genotype.
Determine how many times each geneotype has been tested as a count.
Output the count to an excel sheet.

'''

'\nNeed to determine how many animals have been tested for each gentoype.\nLoad the Summary1.xlsx where each sheet has the summary for each experiment.\nLoad the SummarizingResponses.xlsx that contains a list of each of each cross:genotype.\nDetermine how many times each geneotype has been tested as a count.\nOutput the count to an excel sheet.\n\n'

In [5]:
from pathlib import Path
import pandas as pd
import pathlib

In [6]:
def concat_Excel_file(path_excel):
    #concatenate excel sheets into a single pandas dataframe
    #path_excel = path the excel file
    xl = pd.ExcelFile(path_excel)
    #print(xl.sheet_names)
    exceldata=xl.parse(xl.sheet_names[0])
    exceldata['Sheet'] = xl.sheet_names[0]
    for sheet in xl.sheet_names[1:]:
        cframe = xl.parse(sheet, encoding='utf-8')
        cframe['Sheet'] = sheet
        exceldata = exceldata.append(cframe)
    #change the index so there are no repeatd numbers
    newindex = range(0, len(exceldata))
    exceldata.index = newindex
    return exceldata


In [9]:
summaryfile = '/media/daniel/Seagate5TB/A83/A83_data4/A83_Data3_Summary.xlsx'
experimental_sum = concat_Excel_file(summaryfile)
print(experimental_sum['Date'].unique())


[20190312 20190311]


In [10]:
genotypes = '/media/daniel/Seagate5TB/A83/A83_data4/Analysis20190313/SummarizingResponses.xlsx'
genotypes = concat_Excel_file(genotypes)
genotypes.dropna(axis=0, subset = ['Cross'], inplace = True)

In [11]:
genotypes.columns

Index(['Cross', 'Driver Line', 'Respond', 'Total', 'Description', 'Sheet'], dtype='object')

In [12]:
#add cross designation to each row in exceldata
#need to remove empyt rows contaning nan in 'Sample Name' column
experimental_sum.dropna(axis = 0, subset = ['Sample Name'], inplace = True)
experimental_sum['Cross'] = ''
for row, dseries in genotypes.iterrows():
    ccross = dseries['Cross']
    for row2, dseries2 in experimental_sum.iterrows():
        if ccross in dseries2['Sample Name']:
            experimental_sum['Cross'].loc[row2] = ccross
            experimental_sum['Genotype'].loc[row2] = dseries['Driver Line']

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
#select for only regular saline
print(experimental_sum['Stim Protocol'].unique())
experimental_sum = experimental_sum[experimental_sum['Stim Protocol'] == 'stimDur300at30, stimDur300at15']
print(experimental_sum['Stim Protocol'].unique())

['stimDur300at30, stimDur300at15']
['stimDur300at30, stimDur300at15']


In [14]:
#get counts and save to excel file
targetfolder = str(pathlib.Path.cwd().parents[0] / 'Counts_Tested.xlsx')
groups = experimental_sum.groupby(['Cross', 'Genotype'])
groups[['Cross']].count().to_excel(targetfolder)

In [15]:
#convert counts to dictionary to add back to summarizingresponses.xlsx
counts =groups[['Cross']].count()
dict_count = counts.to_dict()
dict_count2 = {}
for ckey in dict_count['Cross']:
    dict_count2[ckey[0]] = dict_count['Cross'][ckey]
    
genotypes['Number Tested'] =''
for row, dseries, in genotypes.iterrows():
    if dseries['Cross'] in dict_count2:
        genotypes['Number Tested'].loc[row] = dict_count2[dseries['Cross']] 
        
    

In [17]:
#drop columns and output as an excel file
genotypes.drop(['Respond', 'Total', 'Description'], axis=1)
groups = genotypes.groupby('Sheet')
output = '/media/daniel/Seagate5TB/A83/A83_data4/Analysis20190313/SummarizingResponses_Output.xlsx'
writer = pd.ExcelWriter(output, engine='xlsxwriter')
for cgroup, dseries in groups:
    print(cgroup)
    dseries.to_excel(writer, sheet_name = cgroup)
writer.save()

Downstream
Upstream
